In [0]:
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install transformers

In [0]:
import math
import pickle
import random

import numpy as np
from sklearn.metrics import classification_report 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader, Sampler
from torch.nn.utils.rnn import pad_sequence
import transformers
from transformers import XLMRobertaModel, XLMRobertaTokenizer


train_data_dir = '/content/drive/My Drive/Colab Notebooks/ezafe/data/bijankhan_corpus.tsv'
pickled_data_dir = '/content/drive/My Drive/Colab Notebooks/ezafe/data/roberta_data.pickle'
model_dir = '/content/drive/My Drive/Colab Notebooks/ezafe/bert_model_pos/model.pickle'
last_i_dir = '/content/drive/My Drive/Colab Notebooks/ezafe/bert_model_pos/last_i'

HIDDEN_SIZE = 768
NUM_EPOCHS = 100
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
DROPOUT_RATE = .1
BEGIN_TOKEN = 0
END_TOKEN = 2
PAD_TOKEN = 1
TAG_PAD_OR_MASK_TOKEN = 14

random.seed(17)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

In [0]:
def data_reader(directory):
    sents, sent = [], [BEGIN_TOKEN]
    all_pos_tags, pos_tags = [], [TAG_PAD_OR_MASK_TOKEN]
    pos_to_id = {}
    with open(directory) as bijankhan_corpus:
        for line in bijankhan_corpus:
            if line != '\n':
                word, pos_tag, ezafe_tag = line.strip().split('\t')

                word = word.replace('ي', 'ی').replace('ك', 'ک').replace('ة', 'ه')
                word = tokenizer.encode(word, add_special_tokens=False)
                sent.extend(word)

                if pos_tag not in pos_to_id:
                    pos_to_id[pos_tag] = len(pos_to_id)

                pos_tags.extend([pos_to_id[pos_tag]] + [TAG_PAD_OR_MASK_TOKEN for _ in range(len(word) - 1)])
            else:
                assert len(sent) == len(pos_tags)
                sents.append(sent + [END_TOKEN])
                all_pos_tags.append(pos_tags + [TAG_PAD_OR_MASK_TOKEN])
                
                sent = [BEGIN_TOKEN]
                pos_tags = [TAG_PAD_OR_MASK_TOKEN]

    return sents, all_pos_tags


class MySampler(Sampler):
    def __init__(self, data, i=0):
        self.seq = list(range(len(data)))[i * BATCH_SIZE:]
        random.shuffle(self.seq)

    def __iter__(self):
        return iter(self.seq)

    def __len__(self):
        return len(self.seq)


class POSDataset(Dataset):
    def __init__(self, data):
       self.samples = data

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        x = torch.tensor(sample[0])
        mask = torch.sign(x)
        y = torch.tensor(sample[1])

        return x, mask, y

def collate_fn(batch):
    xs, masks, ys = zip(*batch)
    padded_xs = pad_sequence(xs, batch_first=True, padding_value=PAD_TOKEN)
    padded_masks = pad_sequence(masks, batch_first=True)
    padded_ys = pad_sequence(ys, batch_first=True, padding_value=TAG_PAD_OR_MASK_TOKEN)

    return {'input_ids': padded_xs, 'attention_masks': padded_masks}, padded_ys

In [0]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

try:
    with open(pickled_data_dir, 'rb') as out_file:
        sents, ezafe_tags = pickle.load(out_file)
except FileNotFoundError:
    sents, ezafe_tags = data_reader(train_data_dir) 
    with open(pickled_data_dir, 'wb') as out_file:
        pickle.dump([sents, ezafe_tags], out_file)

random.seed(17)
ids = list(range(len(sents)))
random.shuffle(ids)

sents, ezafe_tags = zip(*[(sents[i], ezafe_tags[i]) for i in ids])

sents, ezafe_tags = zip(*[(sent, ezafe) for sent, ezafe in zip(sents, ezafe_tags) if len(sent) <= 512])

data_split_1 = int(len(sents) * .1)
data_split_2 = int(len(sents) * .2)

test_data = list(zip(sents[:data_split_1], ezafe_tags[:data_split_1]))
valid_data = list(zip(sents[data_split_1:data_split_2], ezafe_tags[data_split_1:data_split_2]))
train_data = list(zip(sents[data_split_2:], ezafe_tags[data_split_2:]))

try:
    with open(model_dir + '/last_i') as in_file:
        last_i = int(in_file.read().strip())
except FileNotFoundError:
    last_i = 0

print('last i:', last_i)

train_dataset = POSDataset(train_data)
train_sampler = MySampler(train_dataset, last_i)

valid_dataset = POSDataset(valid_data)

print(len(train_dataset))
print(len(valid_dataset))

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.xlmroberta_model = xlmroberta_model
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.pos_classifier = nn.Linear(HIDDEN_SIZE, 15)

    def forward(self, x, mask):
        sequence_output, _ = self.xlmroberta_model(x, mask)
        sequence_output = self.dropout(sequence_output)
        pos_pred = self.pos_classifier(sequence_output)
        return pos_pred

try:
    model = torch.load(model_dir + str(last_i))    
except:
    xlmroberta_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')
    model = Net().to(device)

In [0]:
train_data_loader = DataLoader(dataset=train_dataset,                                                         
                               batch_size=BATCH_SIZE,                                            
                               collate_fn=collate_fn,
                               sampler=train_sampler,
                               shuffle=False,
                               num_workers=4)

valid_data_loader = DataLoader(dataset=valid_dataset,                                                         
                              batch_size=BATCH_SIZE,                                            
                              collate_fn=collate_fn,
                              shuffle=False,
                              num_workers=4)

criterion = nn.CrossEntropyLoss(ignore_index=TAG_PAD_OR_MASK_TOKEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    for i, (x, pos_true) in enumerate(train_data_loader):
        i += last_i

        optimizer.zero_grad()

        pos_pred = model(x['input_ids'].to(device), x['attention_masks'].to(device))

        loss = criterion(pos_pred.view(-1, pos_pred.shape[-1]).cpu(), pos_true.view(-1))
 
        loss.backward()
        optimizer.step()
 
        if i % 2500 == 0 and (epoch != 0 or i >= 10_000):
            print('testing...')
            # ezafe_true_labels, ezafe_pred_labels = [], []
            pos_true_labels, pos_pred_labels = [], []            
            for x, pos_true in valid_data_loader:
                pos_pred = model(x['input_ids'].to(device), x['attention_masks'].to(device))     
                pos_true_labels.extend(pos_true.reshape(-1).numpy().tolist())
                pos_pred_labels.extend(torch.argmax(pos_pred, -1).reshape(-1).cpu().numpy().tolist())
 
            pos_true_labels_, pos_pred_labels_ = np.array(pos_true_labels), np.array(pos_pred_labels)
 
            ezafe_pos_mask_or_pad_ids = np.where(pos_true_labels_ == TAG_PAD_OR_MASK_TOKEN)[0]

            pos_true_labels_ = np.delete(pos_true_labels_, ezafe_pos_mask_or_pad_ids)
            pos_pred_labels_ = np.delete(pos_pred_labels_, ezafe_pos_mask_or_pad_ids)
 
            pos_f1 = classification_report(pos_true_labels_, pos_pred_labels_, digits=4)
          
            summary = f'epoch: {epoch + 1} | step: {i} | POS_F1:\n{pos_f1}\n\n'
            print(summary)
                            
            # save everything
            with open(model_dir + 'summary.txt', 'a+') as out_file:
                out_file.write(summary + '\n')           
                 
            with open(model_dir + 'last_i', 'w+') as out_file:
                out_file.write(str(i))          

            os.mkdir(model_dir + str(i))
            torch.save(model, model_dir + str(i))

In [0]:
# testing
test_dataset = POSDataset(test_data)

test_data_loader = DataLoader(dataset=test_dataset,                                                         
                              batch_size=BATCH_SIZE,                                            
                              collate_fn=collate_fn,
                              shuffle=False,
                              num_workers=4)

i = 17500
model = model = XLMRobertaForTokenClassification.from_pretrained(model_dir + str(i))

model.to(device)

for x, y_true in test_data_loader:
    y_true = y_true.to(device)

    y_pred = model(x['input_ids'].to(device), x['attention_masks'].to(device))[0]

    test_true_labels.extend(y_true.reshape(-1).cpu().numpy().tolist())
    test_pred_labels.extend(torch.argmax(y_pred, -1).reshape(-1).cpu().numpy().tolist())

test_true_labels_, test_pred_labels_ = np.array(test_true_labels), np.array(test_pred_labels)
test_pad_mask_ids = np.where(test_true_labels_ == TAG_PAD_OR_MASK_TOKEN)[0]
test_true_labels_ = np.delete(test_true_labels_, test_pad_mask_ids)
test_pred_labels_ = np.delete(test_pred_labels_, test_pad_mask_ids)

test_f1 = classification_report(test_true_labels_, test_pred_labels_, digits=4)

summary = f'epoch: {epoch + 1} | step: {i}:\n\n{test_f1}\n'
print(summary)